# Twitter corpus creation and LDA topic modelling

## Introduction

This notebook has two sections:
1. Prototype code for creating a gensim-compatible corpus from my collection of tweets.
2. Training an LDA topic model on a subset of the corpus.

This is largely prototyping and experimenting with model hyperparameters. When done, I'll create separate scripts for each part of this. There's a very rudimentary version of this (probably not committed. Oops...) using a Wikipedia corpus for training, but I wasn't satisfied with it and the preprocessing and retraining takes about 12 hours.

## Libraries and setup

In [11]:
# Python libs
import sys, os
from dotenv import find_dotenv, load_dotenv
import re
import logging
from pathlib import Path
from pprint import pprint
import random

# Database
import pymongo

# NLP libs
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

from gensim.test.utils import datapath
from gensim.utils import simple_preprocess
from gensim.test.utils import common_texts
from gensim.models import TfidfModel, CoherenceModel, LdaMulticore
from gensim.models.ldamodel import LdaModel
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser
from gensim.corpora import Dictionary, MmCorpus

from sklearn.model_selection import ParameterGrid

# Visualisation libs
import matplotlib.pyplot as plt

In [12]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Data loading

In [13]:
# From src/data/db_handlers.py
def mongodb_connect():
    """
    Establish connection to MongoDB
    db name given in .env file
    """
    dotenv_path = find_dotenv()
    load_dotenv(dotenv_path)

    client = pymongo.MongoClient(os.environ.get("DATABASE_URL"), 27017)
    db = client.tweetbase
    return db

In [14]:
db = mongodb_connect()

The Tweetbase db contains two separate collections of tweets, one from Aberdeen, Scotland and the second from Hammersmith, London.

In [15]:
print(f"""There are {db.tweets_abdn.count()} tweets from Aberdeen 
and {db.tweets_hsmith.count()} tweets from Hammersmith in the db""")

There are 164523 tweets from Aberdeen 
and 160335 tweets from Hammersmith in the db


In [16]:
# Get full_text field for each tweet
# Only the first 50 tweets were returned for testing purposes
tweets_abdn = db.tweets_abdn.find({}, {"_id": 0, "full_text": 1})[:50000]
tweets_hsmith = db.tweets_hsmith.find({}, {"_id": 0, "full_text": 1})[:50000]

# Convert mongodb cursor objects into lists
tweets_abdn = [_.get("full_text") for _ in tweets_abdn]
tweets_hsmith = [_.get("full_text") for _ in tweets_hsmith]

In [17]:
tweets = tweets_abdn + tweets_hsmith

In [18]:
# Get indices for a random sample of 50 tweets for inspection
tweets_sample_idxs = [idx for idx, _ in random.sample(list(enumerate(tweets)), 50)]
pprint([tweets[i] for i in tweets_sample_idxs])

['Duit habis, novel yg kutunggu h-3.\n\nMau nangis saya.',
 'Left Billy only 8 hours ago and miss him already, sad times\U0001f97a',
 '@RedMHiro @elgatogaming It runs so well! No delay from my Xbox to pc and '
 'doesn’t even use 10% CPU. It’s fantastic!',
 'Are you a #helicopter #pilot in either #transport or #SAR #searchandrescue '
 'operations with an interest in progressing #CRM? Then my research needs you. '
 'Please drop me a message to take part in our most recent study exploring '
 'pilot skills and the factors that influence them.',
 '@mintR_TV Amalgamations are fine if you personally know the other 2 '
 'streamers but id suggest keeping to yourself. By all means advertise for '
 'each other though.',
 'How do you calm sensitive skin?\n'
 '\n'
 'Cleansing is a crucial morning and evening ritual for your skin all year '
 "round, especially if it's sensitive. It's a vital first step and ensures "
 'that… https://t.co/UadSb90zy5',
 '@kettls @OutsideXbox Been meaning to go since it

## Corpus preprocessing

### Cleaning

In [19]:
# Remove links and @ prefixes from tweets
tweets = [re.sub('@|https?\://\S+', '', t) for t in tweets]

In [20]:
pprint([tweets[i] for i in tweets_sample_idxs])

['Duit habis, novel yg kutunggu h-3.\n\nMau nangis saya.',
 'Left Billy only 8 hours ago and miss him already, sad times\U0001f97a',
 'RedMHiro elgatogaming It runs so well! No delay from my Xbox to pc and '
 'doesn’t even use 10% CPU. It’s fantastic!',
 'Are you a #helicopter #pilot in either #transport or #SAR #searchandrescue '
 'operations with an interest in progressing #CRM? Then my research needs you. '
 'Please drop me a message to take part in our most recent study exploring '
 'pilot skills and the factors that influence them.',
 'mintR_TV Amalgamations are fine if you personally know the other 2 streamers '
 'but id suggest keeping to yourself. By all means advertise for each other '
 'though.',
 'How do you calm sensitive skin?\n'
 '\n'
 'Cleansing is a crucial morning and evening ritual for your skin all year '
 "round, especially if it's sensitive. It's a vital first step and ensures "
 'that… ',
 'kettls OutsideXbox Been meaning to go since it opened! Dundee have pumped 

### Tokenizing

Tokenization of the tweets was performed with `gensim.utils.simple_preprocess()`. This method only produces unigram tokens. Using `gensim.models.phrases.Phraser` on the tokenized output should create bigrams, but it is unclear at present whether the method used below actually did for the given input...

In [21]:
tweets_tokens = [simple_preprocess(t) for t in tweets]

In [22]:
sentences = Text8Corpus(datapath('testcorpus.txt'))
phrases = Phrases(sentences, min_count=1, threshold=1)

# bigram = Phrases(common_texts)
bigram = Phraser(phrases)
tweets_tokens = [bigram[t] for t in tweets_tokens]

2019-07-18 11:01:42,897 : INFO : collecting all words and their counts
2019-07-18 11:01:42,898 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-07-18 11:01:42,899 : INFO : collected 37 word types from a corpus of 29 words (unigram + bigrams) and 1 sentences
2019-07-18 11:01:42,899 : INFO : using 37 counts as vocab in Phrases<0 vocab, min_count=1, threshold=1, max_vocab_size=40000000>
2019-07-18 11:01:42,900 : INFO : source_vocab length 37
2019-07-18 11:01:42,900 : INFO : Phraser built with 3 phrasegrams


In [23]:
pprint([tweets_tokens[i] for i in tweets_sample_idxs])

[['duit', 'habis', 'novel', 'yg', 'kutunggu', 'mau', 'nangis', 'saya'],
 ['left',
  'billy',
  'only',
  'hours',
  'ago',
  'and',
  'miss',
  'him',
  'already',
  'sad',
  'times'],
 ['redmhiro',
  'elgatogaming',
  'it',
  'runs',
  'so',
  'well',
  'no',
  'delay',
  'from',
  'my',
  'xbox',
  'to',
  'pc',
  'and',
  'doesn',
  'even',
  'use',
  'cpu',
  'it',
  'fantastic'],
 ['are',
  'you',
  'helicopter',
  'pilot',
  'in',
  'either',
  'transport',
  'or',
  'sar',
  'searchandrescue',
  'operations',
  'with',
  'an',
  'interest',
  'in',
  'progressing',
  'crm',
  'then',
  'my',
  'research',
  'needs',
  'you',
  'please',
  'drop',
  'me',
  'message',
  'to',
  'take',
  'part',
  'in',
  'our',
  'most',
  'recent',
  'study',
  'exploring',
  'pilot',
  'skills',
  'and',
  'the',
  'factors',
  'that',
  'influence',
  'them'],
 ['mintr_tv',
  'amalgamations',
  'are',
  'fine',
  'if',
  'you',
  'personally',
  'know',
  'the',
  'other',
  'streamers',
  'b

### Stopword removal

In [24]:
stop = stopwords.words('english')
print(stop)
whitelist = []

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

For some reason negated forms of *should*, *would* and *might* are included, but not the regular forms. I've added them to the list myself.

In [25]:
stop_additions = ['should', 'would', 'might', 'could']
stop = stop + stop_additions

In [26]:
whitelist = []

tweets_tokens = [[word for word in sentence if word in whitelist or word not in stop]
     for sentence in tweets_tokens]

In [27]:
pprint([tweets_tokens[i] for i in tweets_sample_idxs])

[['duit', 'habis', 'novel', 'yg', 'kutunggu', 'mau', 'nangis', 'saya'],
 ['left', 'billy', 'hours', 'ago', 'miss', 'already', 'sad', 'times'],
 ['redmhiro',
  'elgatogaming',
  'runs',
  'well',
  'delay',
  'xbox',
  'pc',
  'even',
  'use',
  'cpu',
  'fantastic'],
 ['helicopter',
  'pilot',
  'either',
  'transport',
  'sar',
  'searchandrescue',
  'operations',
  'interest',
  'progressing',
  'crm',
  'research',
  'needs',
  'please',
  'drop',
  'message',
  'take',
  'part',
  'recent',
  'study',
  'exploring',
  'pilot',
  'skills',
  'factors',
  'influence'],
 ['mintr_tv',
  'amalgamations',
  'fine',
  'personally',
  'know',
  'streamers',
  'id',
  'suggest',
  'keeping',
  'means',
  'advertise',
  'though'],
 ['calm',
  'sensitive',
  'skin',
  'cleansing',
  'crucial',
  'morning',
  'evening',
  'ritual',
  'skin',
  'year',
  'round',
  'especially',
  'sensitive',
  'vital',
  'first',
  'step',
  'ensures'],
 ['kettls',
  'outsidexbox',
  'meaning',
  'go',
  'sin

### Lemmatizer

Lemmatization is grouping words under their lemma, or dictionary form e.g. *knows* and *knew* under *know*, or *feet* under *foot*. This requires knowledge of the Part of Speech (PoS) of the item.

While lemmatization was performed on the corpus, it should be noted that it may not necessarily be beneficial. [Schofield and Mimno (2016)](https://www.mitpressjournals.org/doi/abs/10.1162/tacl_a_00099) report that, at best, text preprocessed with the NLTK WordNet lemmatizer showed no meaningful change in topic coherence scores when it comes to LDA topic modelling compared with data that had not been stemmed. At worst, some stemming methods decrease LDA topic stability.

TODO: Possibly compare a lemmatized and unlemmatized version of the corpus

In [28]:
# From https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [29]:
wnl = WordNetLemmatizer()

tweets_tokens = [[wnl.lemmatize(word, get_wordnet_pos(word)) for word in sentence]
    for sentence in tweets_tokens]

In [30]:
pprint([tweets_tokens[i] for i in tweets_sample_idxs])

[['duit', 'habis', 'novel', 'yg', 'kutunggu', 'mau', 'nangis', 'saya'],
 ['left', 'billy', 'hour', 'ago', 'miss', 'already', 'sad', 'time'],
 ['redmhiro',
  'elgatogaming',
  'run',
  'well',
  'delay',
  'xbox',
  'pc',
  'even',
  'use',
  'cpu',
  'fantastic'],
 ['helicopter',
  'pilot',
  'either',
  'transport',
  'sar',
  'searchandrescue',
  'operation',
  'interest',
  'progress',
  'crm',
  'research',
  'need',
  'please',
  'drop',
  'message',
  'take',
  'part',
  'recent',
  'study',
  'explore',
  'pilot',
  'skill',
  'factor',
  'influence'],
 ['mintr_tv',
  'amalgamation',
  'fine',
  'personally',
  'know',
  'streamer',
  'id',
  'suggest',
  'keep',
  'mean',
  'advertise',
  'though'],
 ['calm',
  'sensitive',
  'skin',
  'cleanse',
  'crucial',
  'morning',
  'even',
  'ritual',
  'skin',
  'year',
  'round',
  'especially',
  'sensitive',
  'vital',
  'first',
  'step',
  'ensures'],
 ['kettls',
  'outsidexbox',
  'meaning',
  'go',
  'since',
  'open',
  'dunde

In [31]:
set_tmp = set()
for sentence in tweets_tokens:
    for word in sentence:
        set_tmp.add(word)
print(f"There are {len(set_tmp)} unique words in the corpus")
print(f"The first 150 words are:\n {list(set_tmp)[:150]}")

There are 136724 unique words in the corpus
The first 150 words are:
 ['hsn_ron', 'opang', 'markcooke', 'shanklanding', 'ladycorvia', 'rwsreece', 'nativeesoul', 'шутить', 'uisgebeatha', 'ماله', 'ira', 'он', 'toweroflondon', 'sheilamarson', 'свой', 'murghi', 'imjames_', 'korzystasz', 'unexpectedly', 'portobelloshack', 'villager', 'yet', 'shree_das', 'clauding', 'بيه', 'شاهد', 'bandung', 'lewiskingy', 'shona', 'leigh__h', 'ulovey', 'skeletalmess', 'colombian', 'ماعرفناه', 'fflad', 'restmarkinnov', 'nigra', 'noclass', 'bertahan', '표출하려', 'trudesundset', 'babettewasserma', 'حرفا', 'dino_melaye', 'gossipnya', 'stopbrexitnow', 'shayouy', 'floofy', 'گیری', 'tooley_evans', 'elissabairdx', 'anning', 'leslie', 'rightfully', 'colon', 'heyitschloejade', 'azizanalysis', 'hadeert', 'hunker', 'whynstuart', 'mandywolfbear', 'reddingpower', 'старше', 'ведь', 'yorkshiredalesb', 'emily_hutton_', 'chantelsophiaxo', 'malcom', '国際', 'melodic', 'finalmente', 'vvv_rfc', 'сухостоем', 'dunked', 'jordanbpeterson

Many of the words in the corpus so far are Twitter username mentions and hashtags. The creation of the dictionary in the next section will filter those that are not widely used.

## Creating Dictionary and BoW corpus

In [32]:
KEEP_WORDS = 100000 # Max number of words in dictionary
CORPUS_PATH = Path('../../data/corpora') # Location to save corpus and dict

In [33]:
dictionary = Dictionary(tweets_tokens)

# Filter the dictionary:
#    Words must appear in no fewer than n documents
#    Words must not appear in more than n of the total documents
dictionary.filter_extremes(no_below=40, no_above=0.05, keep_n=KEEP_WORDS)
print(dictionary.token2id)

2019-07-18 11:03:59,366 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-07-18 11:03:59,511 : INFO : adding document #10000 to Dictionary(19563 unique tokens: ['best', 'friday', 'night', 'thing', 'add']...)
2019-07-18 11:03:59,657 : INFO : adding document #20000 to Dictionary(31740 unique tokens: ['best', 'friday', 'night', 'thing', 'add']...)
2019-07-18 11:03:59,804 : INFO : adding document #30000 to Dictionary(42304 unique tokens: ['best', 'friday', 'night', 'thing', 'add']...)
2019-07-18 11:03:59,953 : INFO : adding document #40000 to Dictionary(51777 unique tokens: ['best', 'friday', 'night', 'thing', 'add']...)
2019-07-18 11:04:00,109 : INFO : adding document #50000 to Dictionary(60635 unique tokens: ['best', 'friday', 'night', 'thing', 'add']...)
2019-07-18 11:04:00,258 : INFO : adding document #60000 to Dictionary(79663 unique tokens: ['best', 'friday', 'night', 'thing', 'add']...)
2019-07-18 11:04:00,415 : INFO : adding document #70000 to Dictionary(95875 uni

{'best': 0, 'friday': 1, 'night': 2, 'thing': 3, 'add': 4, 'chip': 5, 'day': 6, 'flat': 7, 'fry': 8, 'meal': 9, 'pm': 10, 'ring': 11, 'sauce': 12, 'serve': 13, 'till': 14, 'weekend': 15, 'cool': 16, 'look': 17, 'pocket': 18, 'te': 19, 'top': 20, 'bad': 21, 'gorgeous': 22, 'close': 23, 'definitely': 24, 'drive': 25, 'fact': 26, 'know': 27, 'left': 28, 'lucky': 29, 'onto': 30, 'right': 31, 'street': 32, 'union': 33, 'begin': 34, 'cold': 35, 'time': 36, 'good': 37, 'blue': 38, 'call': 39, 'home': 40, 'order': 41, 'pre': 42, 'road': 43, 'need': 44, 'people': 45, 'reason': 46, 'bother': 47, 'pay': 48, 'point': 49, 'question': 50, 'dont': 51, 'mix': 52, 'politics': 53, 'sport': 54, 'always': 55, 'base': 56, 'company': 57, 'enter': 58, 'giveaway': 59, 'global': 60, 'prize': 61, 'range': 62, 'say': 63, 'since': 64, 'st': 65, 'uk': 66, 'way': 67, 'amp': 68, 'background': 69, 'expose': 70, 'fight': 71, 'gang': 72, 'hate': 73, 'heard': 74, 'never': 75, 'state': 76, 'strike': 77, 'white': 78, 'arg

Along with the LDA model hyperparameters, the dictionary filtering above will have a considerable impact on the topic model.

In [34]:
# Create the corpus
tweet_corpus = [dictionary.doc2bow(tweet) for tweet in tweets_tokens] # Use the dict to create BoW vectors
# tweet_corpus = [text for text in tweet_corpus if len(text) > 0]
MmCorpus.serialize(str(CORPUS_PATH / 'tweets') + '_bow.mm', tweet_corpus, progress_cnt=10000)

2019-07-18 11:04:01,939 : INFO : storing corpus in Matrix Market format to ../../data/corpora/tweets_bow.mm
2019-07-18 11:04:01,940 : INFO : saving sparse matrix to ../../data/corpora/tweets_bow.mm
2019-07-18 11:04:01,940 : INFO : PROGRESS: saving document #0
2019-07-18 11:04:02,044 : INFO : PROGRESS: saving document #10000
2019-07-18 11:04:02,143 : INFO : PROGRESS: saving document #20000
2019-07-18 11:04:02,243 : INFO : PROGRESS: saving document #30000
2019-07-18 11:04:02,340 : INFO : PROGRESS: saving document #40000
2019-07-18 11:04:02,443 : INFO : PROGRESS: saving document #50000
2019-07-18 11:04:02,525 : INFO : PROGRESS: saving document #60000
2019-07-18 11:04:02,609 : INFO : PROGRESS: saving document #70000
2019-07-18 11:04:02,690 : INFO : PROGRESS: saving document #80000
2019-07-18 11:04:02,774 : INFO : PROGRESS: saving document #90000
2019-07-18 11:04:02,858 : INFO : saved 100000x2717 matrix, density=0.178% (482349/271700000)
2019-07-18 11:04:02,859 : INFO : saving MmCorpus inde

In [35]:
dictionary.save_as_text(str(CORPUS_PATH / 'tweets') + '_wordids.txt.bz2')

2019-07-18 11:04:02,867 : INFO : saving dictionary mapping to ../../data/corpora/tweets_wordids.txt.bz2


## Create LDA topic model

In [36]:
CORPUS_PATH = Path('../../data/corpora') # Location to save corpus and dict
MODEL_PATH = Path('../../models')

In [37]:
mm = MmCorpus(str(CORPUS_PATH / 'tweets') + '_bow.mm')

2019-07-18 11:04:03,150 : INFO : loaded corpus index from ../../data/corpora/tweets_bow.mm.index
2019-07-18 11:04:03,151 : INFO : initializing cython corpus reader from ../../data/corpora/tweets_bow.mm
2019-07-18 11:04:03,151 : INFO : accepted corpus with 100000 documents, 2717 features, 482349 non-zero entries


In [38]:
dictionary = Dictionary.load_from_text(str(CORPUS_PATH / 'tweets') + '_wordids.txt.bz2')

### FInd good parameters

Find LDA model hyperparamters that give the highest topic coherence score.

In [51]:
parameter_grid = ParameterGrid({
    'num_topics': [100, 150, 200],
    'chunksize': [10000],
    'passes': [3, 5],
    'iterations': [25],
    'decay': [0.5, 0.6],
    'eval_every': [None]
})

In [52]:
lda_models = []
coherence = []
parameters = []

for params in parameter_grid:
    parameters.append(params)
    
    # Fit the model
    lda = LdaMulticore(corpus=mm, id2word=dictionary,
                   **params,
                   workers=3)
    
    # Get the coherence score for the model
    cm = CoherenceModel(model=lda, texts=tweets_tokens, dictionary=dictionary, coherence='c_v')
    
    # Append results
    lda_models.append(lda)
    coherence.append(cm.get_coherence())

2019-07-18 11:21:58,094 : INFO : using symmetric alpha at 0.01
2019-07-18 11:21:58,095 : INFO : using symmetric eta at 0.01
2019-07-18 11:21:58,096 : INFO : using serial LDA version on this node
2019-07-18 11:21:58,133 : INFO : running online LDA training, 100 topics, 3 passes over the supplied corpus of 100000 documents, updating every 30000 documents, evaluating every ~0 documents, iterating 25x with a convergence threshold of 0.001000
2019-07-18 11:21:58,138 : INFO : training LDA model using 3 processes
2019-07-18 11:21:58,213 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #10000/100000, outstanding queue size 1
2019-07-18 11:21:58,269 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #20000/100000, outstanding queue size 2
2019-07-18 11:21:58,318 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #30000/100000, outstanding queue size 3
2019-07-18 11:21:58,383 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #40000/10000

2019-07-18 11:22:14,855 : INFO : topic #58 (0.010): 0.037*"ada" + 0.018*"tak" + 0.017*"kalau" + 0.012*"london" + 0.011*"work" + 0.011*"go" + 0.010*"excite" + 0.010*"see" + 0.008*"dah" + 0.008*"dapat"
2019-07-18 11:22:14,857 : INFO : topic #66 (0.010): 0.027*"aberdeen" + 0.025*"job" + 0.019*"ya" + 0.018*"hire" + 0.018*"scotland" + 0.015*"time" + 0.015*"careerarc" + 0.014*"music" + 0.014*"road" + 0.010*"council"
2019-07-18 11:22:14,858 : INFO : topic #0 (0.010): 0.023*"go" + 0.018*"want" + 0.015*"use" + 0.014*"time" + 0.011*"مع" + 0.011*"لا" + 0.009*"say" + 0.008*"work" + 0.008*"know" + 0.008*"فيه"
2019-07-18 11:22:14,861 : INFO : topic #96 (0.010): 0.060*"yes" + 0.029*"happy" + 0.023*"birthday" + 0.019*"take" + 0.011*"please" + 0.011*"go" + 0.010*"wonder" + 0.008*"one" + 0.008*"u" + 0.007*"decent"
2019-07-18 11:22:14,872 : INFO : topic diff=inf, rho=0.288675
2019-07-18 11:22:18,012 : INFO : merging changes from 30000 documents into a model of 100000 documents
2019-07-18 11:22:18,057 : I

2019-07-18 11:22:30,832 : INFO : topic #0 (0.010): 0.033*"winner" + 0.032*"use" + 0.031*"لا" + 0.023*"bottle" + 0.023*"orang" + 0.021*"brighton" + 0.020*"مع" + 0.018*"فيه" + 0.018*"go" + 0.015*"everywhere"
2019-07-18 11:22:30,832 : INFO : topic #51 (0.010): 0.041*"bus" + 0.039*"finally" + 0.025*"man" + 0.024*"wanna" + 0.023*"track" + 0.020*"cover" + 0.019*"human" + 0.018*"town" + 0.017*"driver" + 0.017*"time"
2019-07-18 11:22:30,833 : INFO : topic #18 (0.010): 0.159*"de" + 0.117*"la" + 0.088*"que" + 0.058*"en" + 0.043*"un" + 0.037*"tu" + 0.023*"e" + 0.020*"ça" + 0.017*"por" + 0.016*"al"
2019-07-18 11:22:30,834 : INFO : topic #72 (0.010): 0.094*"happy" + 0.090*"christmas" + 0.071*"best" + 0.052*"birthday" + 0.032*"day" + 0.025*"omg" + 0.021*"hope" + 0.018*"xxx" + 0.017*"one" + 0.014*"celebrate"
2019-07-18 11:22:30,834 : INFO : topic #50 (0.010): 0.109*"le" + 0.059*"pa" + 0.048*"est" + 0.048*"il" + 0.039*"et" + 0.030*"se" + 0.027*"nah" + 0.025*"ffs" + 0.024*"son" + 0.022*"super"
2019-07-

2019-07-18 11:22:59,587 : INFO : topic diff=inf, rho=0.377964
2019-07-18 11:23:01,732 : INFO : merging changes from 10000 documents into a model of 100000 documents
2019-07-18 11:23:01,766 : INFO : topic #33 (0.010): 0.019*"love" + 0.013*"show" + 0.013*"chelsea" + 0.011*"say" + 0.011*"go" + 0.010*"one" + 0.009*"left" + 0.009*"happy" + 0.008*"watch" + 0.008*"em"
2019-07-18 11:23:01,767 : INFO : topic #10 (0.010): 0.017*"london" + 0.017*"thanks" + 0.014*"shot" + 0.012*"history" + 0.011*"one" + 0.010*"madrid" + 0.010*"natural" + 0.010*"two" + 0.010*"never" + 0.009*"like"
2019-07-18 11:23:01,768 : INFO : topic #28 (0.010): 0.023*"like" + 0.012*"year" + 0.010*"one" + 0.010*"say" + 0.009*"thanks" + 0.008*"como" + 0.008*"happy" + 0.008*"look" + 0.008*"chelsea" + 0.007*"great"
2019-07-18 11:23:01,769 : INFO : topic #5 (0.010): 0.015*"travel" + 0.014*"minute" + 0.011*"one" + 0.009*"shirt" + 0.008*"road" + 0.008*"anyway" + 0.008*"amp" + 0.008*"time" + 0.008*"old" + 0.008*"good"
2019-07-18 11:23:

2019-07-18 11:23:13,288 : INFO : PROGRESS: pass 2, dispatched chunk #7 = documents up to #80000/100000, outstanding queue size 8
2019-07-18 11:23:13,327 : INFO : PROGRESS: pass 2, dispatched chunk #8 = documents up to #90000/100000, outstanding queue size 9
2019-07-18 11:23:15,749 : INFO : PROGRESS: pass 2, dispatched chunk #9 = documents up to #100000/100000, outstanding queue size 9
2019-07-18 11:23:16,012 : INFO : merging changes from 30000 documents into a model of 100000 documents
2019-07-18 11:23:16,052 : INFO : topic #79 (0.010): 0.055*"thanks" + 0.048*"music" + 0.046*"dah" + 0.036*"expect" + 0.024*"когда" + 0.021*"health" + 0.017*"roll" + 0.013*"fight" + 0.012*"mental" + 0.011*"today"
2019-07-18 11:23:16,053 : INFO : topic #0 (0.010): 0.127*"de" + 0.085*"la" + 0.072*"que" + 0.049*"en" + 0.046*"yeah" + 0.039*"un" + 0.034*"el" + 0.028*"tu" + 0.023*"apa" + 0.022*"e"
2019-07-18 11:23:16,054 : INFO : topic #69 (0.010): 0.035*"white" + 0.027*"tonight" + 0.024*"proud" + 0.024*"amp" + 

...

2019-07-18 11:33:34,437 : INFO : topic #31 (0.005): 0.230*"man" + 0.054*"fulham" + 0.044*"account" + 0.037*"jadi" + 0.030*"four" + 0.027*"bank" + 0.019*"sih" + 0.017*"balik" + 0.017*"gue" + 0.011*"jim"
2019-07-18 11:33:34,439 : INFO : topic diff=inf, rho=0.214602
2019-07-18 11:33:36,219 : INFO : merging changes from 10000 documents into a model of 100000 documents
2019-07-18 11:33:36,285 : INFO : topic #53 (0.005): 0.050*"wear" + 0.050*"garden" + 0.032*"receive" + 0.031*"google" + 0.030*"branch" + 0.029*"security" + 0.023*"purpose" + 0.021*"limit" + 0.021*"teach" + 0.017*"fantastic"
2019-07-18 11:33:36,286 : INFO : topic #163 (0.005): 0.057*"men" + 0.037*"jorginho" + 0.028*"medical" + 0.025*"doctor" + 0.025*"pretty" + 0.023*"beat" + 0.021*"ross" + 0.019*"people" + 0.017*"play" + 0.017*"commission"
2019-07-18 11:33:36,287 : INFO : topic #66 (0.005): 0.089*"enjoy" + 0.075*"must" + 0.068*"leave" + 0.053*"eat" + 0.043*"score" + 0.016*"head" + 0.015*"go" + 0.014*"time" + 0.013*"alternative"

2019-07-18 11:34:21,871 : INFO : topic #66 (0.005): 0.019*"fuck" + 0.012*"william" + 0.011*"great" + 0.010*"mean" + 0.009*"go" + 0.009*"scotland" + 0.008*"weekend" + 0.007*"cupcake" + 0.007*"holy" + 0.007*"look"
2019-07-18 11:34:21,872 : INFO : topic #110 (0.005): 0.027*"amp" + 0.015*"say" + 0.010*"need" + 0.010*"interview" + 0.008*"sarri" + 0.008*"vote" + 0.008*"still" + 0.007*"london" + 0.007*"deal" + 0.007*"tea"
2019-07-18 11:34:21,873 : INFO : topic diff=inf, rho=0.251189
2019-07-18 11:34:21,906 : INFO : PROGRESS: pass 1, dispatched chunk #0 = documents up to #10000/100000, outstanding queue size 1
2019-07-18 11:34:21,956 : INFO : PROGRESS: pass 1, dispatched chunk #1 = documents up to #20000/100000, outstanding queue size 2
2019-07-18 11:34:22,005 : INFO : PROGRESS: pass 1, dispatched chunk #2 = documents up to #30000/100000, outstanding queue size 3
2019-07-18 11:34:22,266 : INFO : PROGRESS: pass 1, dispatched chunk #3 = documents up to #40000/100000, outstanding queue size 4
201

2019-07-18 11:34:36,990 : INFO : topic #14 (0.005): 0.050*"attack" + 0.032*"ko" + 0.027*"weird" + 0.023*"sea" + 0.019*"unless" + 0.015*"people" + 0.015*"useless" + 0.013*"today" + 0.013*"need" + 0.012*"disagree"
2019-07-18 11:34:37,004 : INFO : topic #2 (0.005): 0.068*"busy" + 0.037*"lane" + 0.034*"felt" + 0.031*"road" + 0.027*"لك" + 0.026*"court" + 0.018*"one" + 0.017*"drunk" + 0.015*"oops" + 0.015*"alcohol"
2019-07-18 11:34:37,006 : INFO : topic #178 (0.005): 0.184*"thanks" + 0.038*"community" + 0.036*"list" + 0.028*"earlier" + 0.025*"great" + 0.017*"pedro" + 0.013*"see" + 0.010*"support" + 0.009*"need" + 0.009*"anna_soubry"
2019-07-18 11:34:37,010 : INFO : topic diff=inf, rho=0.214602
2019-07-18 11:34:39,306 : INFO : merging changes from 30000 documents into a model of 100000 documents
2019-07-18 11:34:39,377 : INFO : topic #118 (0.005): 0.060*"consider" + 0.040*"stick" + 0.026*"chocolate" + 0.022*"ta" + 0.022*"say" + 0.018*"insta" + 0.018*"highlight" + 0.017*"tfl" + 0.015*"like" + 

2019-07-18 11:34:52,011 : INFO : topic #141 (0.005): 0.126*"party" + 0.094*"labour" + 0.046*"charity" + 0.037*"general" + 0.035*"corbyn" + 0.029*"если" + 0.022*"brexit" + 0.017*"people" + 0.017*"primary" + 0.016*"election"
2019-07-18 11:34:52,012 : INFO : topic #110 (0.005): 0.080*"absolute" + 0.065*"interview" + 0.035*"parliament" + 0.032*"amp" + 0.032*"straight" + 0.026*"adam" + 0.026*"crime" + 0.023*"fee" + 0.021*"deal" + 0.020*"threaten"
2019-07-18 11:34:52,013 : INFO : topic #22 (0.005): 0.098*"sign" + 0.054*"announce" + 0.048*"indeed" + 0.045*"leader" + 0.042*"surprised" + 0.041*"com" + 0.036*"scar" + 0.029*"leadership" + 0.028*"effect" + 0.023*"new"
2019-07-18 11:34:52,013 : INFO : topic #105 (0.005): 0.088*"hard" + 0.066*"wear" + 0.049*"often" + 0.047*"forget" + 0.042*"finish" + 0.039*"mi" + 0.025*"medical" + 0.025*"push" + 0.021*"duty" + 0.021*"usa"
2019-07-18 11:34:52,017 : INFO : topic #4 (0.005): 0.154*"job" + 0.052*"aberdeen" + 0.051*"scotland" + 0.050*"hire" + 0.042*"mad"

2019-07-18 11:35:02,648 : INFO : topic #4 (0.005): 0.189*"job" + 0.059*"aberdeen" + 0.055*"scotland" + 0.049*"hire" + 0.049*"mad" + 0.032*"replace" + 0.030*"opening" + 0.028*"recommend" + 0.024*"careerarc" + 0.023*"work"
2019-07-18 11:35:02,649 : INFO : topic diff=inf, rho=0.196945
2019-07-18 11:35:02,726 : INFO : using ParallelWordOccurrenceAccumulator(processes=3, batch_size=64) to estimate probabilities from sliding windows
2019-07-18 11:35:12,865 : INFO : serializing accumulator to return to master...
2019-07-18 11:35:12,867 : INFO : accumulator serialized
2019-07-18 11:35:12,873 : INFO : serializing accumulator to return to master...
2019-07-18 11:35:12,874 : INFO : serializing accumulator to return to master...
2019-07-18 11:35:12,876 : INFO : accumulator serialized
2019-07-18 11:35:12,886 : INFO : accumulator serialized
2019-07-18 11:35:14,163 : INFO : 3 accumulators retrieved from output queue
2019-07-18 11:35:17,015 : INFO : accumulated word occurrence stats for 83787 virtual 

In [53]:
for params in parameter_grid:
    print(params)

{'chunksize': 10000, 'decay': 0.5, 'eval_every': None, 'iterations': 25, 'num_topics': 100, 'passes': 3}
{'chunksize': 10000, 'decay': 0.5, 'eval_every': None, 'iterations': 25, 'num_topics': 100, 'passes': 5}
{'chunksize': 10000, 'decay': 0.5, 'eval_every': None, 'iterations': 25, 'num_topics': 150, 'passes': 3}
{'chunksize': 10000, 'decay': 0.5, 'eval_every': None, 'iterations': 25, 'num_topics': 150, 'passes': 5}
{'chunksize': 10000, 'decay': 0.5, 'eval_every': None, 'iterations': 25, 'num_topics': 200, 'passes': 3}
{'chunksize': 10000, 'decay': 0.5, 'eval_every': None, 'iterations': 25, 'num_topics': 200, 'passes': 5}
{'chunksize': 10000, 'decay': 0.6, 'eval_every': None, 'iterations': 25, 'num_topics': 100, 'passes': 3}
{'chunksize': 10000, 'decay': 0.6, 'eval_every': None, 'iterations': 25, 'num_topics': 100, 'passes': 5}
{'chunksize': 10000, 'decay': 0.6, 'eval_every': None, 'iterations': 25, 'num_topics': 150, 'passes': 3}
{'chunksize': 10000, 'decay': 0.6, 'eval_every': None, 

In [54]:
print(coherence)

[0.4384908645647992, 0.47253372650237063, 0.4441244130162207, 0.45568775728125976, 0.4194790059190659, 0.4214115472435088, 0.44284943857801046, 0.4604311841446061, 0.4318464192434856, 0.44184628807838644, 0.41799105775870626, 0.4280441069151003]


In [55]:
best_model_idx = coherence.index(max(coherence))
print(f"""Params: {parameters[best_model_idx]}
Coherence: {coherence[best_model_idx]}
""")

Params: {'chunksize': 10000, 'decay': 0.5, 'eval_every': None, 'iterations': 25, 'num_topics': 100, 'passes': 5}
Coherence: 0.47253372650237063



In [56]:
lda_models[best_model_idx].save(str(MODEL_PATH / 'tweets') + 'best_lda.model')

2019-07-18 11:40:24,136 : INFO : saving LdaState object under ../../models/tweetsbest_lda.model.state, separately None
2019-07-18 11:40:24,141 : INFO : saved ../../models/tweetsbest_lda.model.state
2019-07-18 11:40:24,144 : INFO : saving LdaMulticore object under ../../models/tweetsbest_lda.model, separately ['expElogbeta', 'sstats']
2019-07-18 11:40:24,144 : INFO : storing np array 'expElogbeta' to ../../models/tweetsbest_lda.model.expElogbeta.npy
2019-07-18 11:40:24,146 : INFO : not storing attribute dispatcher
2019-07-18 11:40:24,147 : INFO : not storing attribute state
2019-07-18 11:40:24,147 : INFO : not storing attribute id2word
2019-07-18 11:40:24,148 : INFO : saved ../../models/tweetsbest_lda.model


### Load best model

In [58]:
best_lda = LdaModel.load(str(MODEL_PATH / 'tweets') + 'best_lda.model')

2019-07-18 11:41:52,442 : INFO : loading LdaModel object from ../../models/tweetsbest_lda.model
2019-07-18 11:41:52,443 : INFO : loading expElogbeta from ../../models/tweetsbest_lda.model.expElogbeta.npy with mmap=None
2019-07-18 11:41:52,444 : INFO : setting ignored attribute dispatcher to None
2019-07-18 11:41:52,445 : INFO : setting ignored attribute state to None
2019-07-18 11:41:52,446 : INFO : setting ignored attribute id2word to None
2019-07-18 11:41:52,446 : INFO : loaded ../../models/tweetsbest_lda.model
2019-07-18 11:41:52,447 : INFO : loading LdaState object from ../../models/tweetsbest_lda.model.state
2019-07-18 11:41:52,451 : INFO : loaded ../../models/tweetsbest_lda.model.state
